### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 3


### Дисциплина: Управление ИТ-сервисами и контентом

##### Студент:  Матюхин Григорий Васильевич
##### Группа:   НПИбд-01-21

## Москва 2024
***

# Введение

В данное лабораторной работе рассматривается Web API социальной сети Вконтакте. В отличие от API сервисов, рассмотренных в предыдущих лабораторноых работах, API VK включает в себя несколько сотен методов, разные варианты авторизации, набор SDK под различные платформы, а также обширную документацию, некоторые части котороЙ оформлены в виде пособий для новичков.

Сайт Вконтакте использует псевдо-RPC API, где методы находятся на отдельных URL, а параметры передаются через query-строку. В результате этого любые операции, которые требуют контента (например, загрузка картинок) создают URL, на который следует отправить этот контент другим способом.

# Выполнение

## Авторизация

Чтобы получить доступ к данным пользователя, нужно получить токен от него. По умолчанию очень малло данных доступно, создадим `scope` запршивающий все, что нужно в ходе лабораторной работы.

In [1]:
scope=0
(( scope|=1<<2 )) # photos
(( scope|=1<<3 )) # audio
(( scope|=1<<16 )) # offline
(( scope|=1<<17 )) # docs
echo "$scope"

196620


Попоробуем с самым простым процессоом авторизации -- Implicit Flow, по сути тот же OAuth.

In [2]:
echo "https://oauth.vk.com/authorize?client_id=51915772&response_type=token&scope=$scope&revoke=1"

https://oauth.vk.com/authorize?client_id=51915772&response_type=token&scope=196620&revoke=1


Это не сработает, максимум что мы можем получить -- это вечный "offline" токен.

In [3]:
echo "https://id.vk.com/auth?redirect_uri=https%3A%2F%2Foauth.vk.com%2Fblank.html&app_id=51915772&response_type=token&scope=$scope"

https://id.vk.com/auth?redirect_uri=https%3A%2F%2Foauth.vk.com%2Fblank.html&app_id=51915772&response_type=token&scope=196620


Более сложный способ это используя VK ID. Здесь надо получить `silent_token` и обменять его на `access_token`. Это работает, но для доступа ко всем интересным данным пользователя (типо фото, видео и файлов) необходимо подтвердить свой аккаунт разработчика. А для этого нужно подать в ВК свои паспортные данные...

In [4]:
echo "https://oauth.vk.com/authorize?client_id=6733081&redirect_uri=blank.html&scope=$scope&response_type=token"

https://oauth.vk.com/authorize?client_id=6733081&redirect_uri=blank.html&scope=196620&response_type=token


Если этого не хотим делать, то надо найти друга, у которого уже есть приложение созданное до новвоведения, требующего паспорта. К счастью, у меня есть такой друг. Теперь мы можем использовать ссылку выше, чтобы получить `access_token` для всего, что нам нужно.

Сохраним его в файл и прочтем в переменную.

In [5]:
TOKEN=$(cat ./token_file)

## Группа методов `users`

### Поиск по произвольному запросу 

In [6]:
curl -Gs https://api.vk.com/method/users.search \
-d v=5.199 \
-d access_token=$TOKEN \
-d q="Матюхин" \
-d count=2 | jq

{
  "response": {
    "count": 11166,
    "items": [
      {
        "id": 517882843,
        "track_code": "5b74709dRvkCXrT65yLCUJmQwFuoxAnSQ13K_a2gIx8Etj3EcekhkBEJ5MHjK8AMoFVNp2PEYdxFXsr9o8ZRdg",
        "first_name": "Pavel",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 329361482,
        "track_code": "8b1a78c5aZnkkozm9aNwfcyFdflATQ86xc6EzMqyzlWWsATwOqQO8PKVgdny9Coo-EP0CYpNZzTDzYTMxNS8PA",
        "first_name": "Grigorii",
        "last_name": "Matiukhin",
        "can_access_closed": true,
        "is_closed": true
      }
    ]
  }
}


### Дополнительные данные

In [7]:
curl -Gs https://api.vk.com/method/users.search \
-d v=5.199 \
-d access_token=$TOKEN \
-d q="Матюхин" \
-d count=2 \
-d fields=about,activities,bdate | jq

{
  "response": {
    "count": 11166,
    "items": [
      {
        "id": 517882843,
        "about": "",
        "activities": "",
        "track_code": "73f93207VeCRuu4iVxLxhj2A5S2W2cXbOlx3vIAfy3DA1aEwN7A2msHq1XZeEP3bDiRdlLGxy905XHey5m2i",
        "first_name": "Pavel",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 329361482,
        "bdate": "1.7.2003",
        "about": "",
        "activities": "",
        "track_code": "6ca003afeR4cLxe50QXulmugEiYhatJunLWu6VK8JjaujemaHjMaZEh-L73RCeTGWAimngYC3Gifta7nNM5P",
        "first_name": "Grigorii",
        "last_name": "Matiukhin",
        "can_access_closed": true,
        "is_closed": true
      }
    ]
  }
}


### Поиск по родному городу

In [8]:
user_id=$(
    curl -Gs https://api.vk.com/method/users.search \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d q="Матюхин" \
    -d count=2 \
    -d hometown=Moscow \
    | tee >(jq >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 5,
    "items": [
      {
        "id": 737822617,
        "track_code": "00c39848sNMQbMpzIc9ADHtvVDYJad1IrlvbFAWX1uH4IUkwbavXulRux0wvxE8GSKy4-btotUaoWNsUC_GkiA",
        "first_name": "Romario",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 2273004,
        "track_code": "343be69eQ6TWLiE7drKLf57m7sBptQkLB6h3DSIJNKGW2DlqPCUkzcYrelom4dp6rCcAOIHTfg8WshJ_LG8",
        "first_name": "Anton",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      }
    ]
  }
}


In [9]:
echo "$user_id"

737822617


### Найдем подписчиков пользователя

In [10]:
curl -Gs https://api.vk.com/method/users.getFollowers \
-d v=5.199 \
-d access_token=$TOKEN \
-d user_id=$user_id | jq

{
  "response": {
    "count": 53,
    "items": [
      707008454,
      649006809,
      750344121,
      754106530,
      502260917,
      596120431,
      675933697,
      845983304,
      615886554,
      666814859,
      525547111,
      745694113,
      822181760,
      523236642,
      640235273,
      544358353,
      668126350,
      728506309,
      564451351,
      539075208,
      303910990,
      815091873,
      699889712,
      593718582,
      741238893,
      745215990,
      486948463,
      291509053,
      492845249,
      601572607,
      80735991,
      706907459,
      36948703,
      735008424,
      565035031,
      34657950,
      38443196,
      169611638,
      380952954,
      642032785,
      222911255,
      111102867,
      461649609,
      435164616,
      543511185,
      493342860,
      833518571,
      368076549,
      846004551,
      724147965,
      500275235,
      18054956,
      37670947
    ]
  }
}


## Группа методов `database`

### Найдем страну по коду

In [11]:
ru_id=$(
    curl -Gs https://api.vk.com/method/database.getCountries \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d code=RU \
    | tee >(jq >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 1,
        "title": "Russia"
      }
    ]
  }
}


In [12]:
echo "$ru_id"

1


### Найдем город по названию

In [13]:
city_id=$(
    curl -Gs https://api.vk.com/method/database.getCities \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d count=2 \
    -d q=Moscow \
    | tee >(jq >&2) \
    | jq ".response.items | map(select(.id == $ru_id)) | .[0].id"
)

{
  "response": {
    "count": 88,
    "items": [
      {
        "id": 1,
        "title": "Moscow"
      },
      {
        "id": 1904692,
        "title": "Moscow",
        "area": "Latah County",
        "region": "Idaho"
      }
    ]
  }
}


In [14]:
echo $city_id

1


### Найдем школу по названию

In [15]:
curl -Gs https://api.vk.com/method/database.getSchools \
-d v=5.199 \
-d access_token=$TOKEN \
-d city_id=1 \
-d q=="Школа № 1" | jq '.response.items[0]'

### Найдем университет по названию

In [16]:
curl -Gs https://api.vk.com/method/database.getUniversities \
-d v=5.199 \
-d access_token=$TOKEN \
-d city_id=1 \
-d q="RUDN" | jq

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 338,
        "title": "РУДН"
      }
    ]
  }
}


## Группа методов `audio`???

Группы методов по работе с аудио и музыкой не существует!

## Группа методов `docs`

### Получим все документы ползователя

In [17]:
curl -Gs https://api.vk.com/method/docs.get \
-d v=5.199 \
-d access_token=$TOKEN \
-d owner_id=329361482 \
-d type=4 \
-d count=2 | jq

{
  "response": {
    "count": 7,
    "items": [
      {
        "id": 608960774,
        "owner_id": 329361482,
        "title": "THE SOLUTION.png",
        "size": 11465,
        "ext": "png",
        "date": 1631211540,
        "type": 4,
        "url": "https://vk.com/doc329361482_608960774?hash=HszBDYNjqFQZHOoVTMhY5D6fr3l9xJxvV7ZPdsunHK0&dl=lKo4qKLhizZO7OEjeR4XDjdAeStt9GCBiaZGu79lsCw&api=1&no_preview=1",
        "preview": {
          "photo": {
            "sizes": [
              {
                "src": "https://sun9-62.userapi.com/c520036/u329361482/d34/-3/m_8f8c8839e5.jpg",
                "width": 130,
                "height": 24,
                "type": "m"
              },
              {
                "src": "https://sun9-9.userapi.com/c520036/u329361482/d34/-3/s_8f8c8839e5.jpg",
                "width": 100,
                "height": 19,
                "type": "s"
              },
              {
                "src": "https://sun9-62.userapi.com/c520036/u329361482/

### Загрузка документов

Сначала надо получить URL, куда загрузить файл, потом загрузить туда файл используя POST, и в конце "привязать" документ к пользователю.

In [18]:
upload_url=$(
    curl -Gs https://api.vk.com/method/docs.getUploadServer \
    -d v=5.199 \
    -d access_token=$TOKEN \
    | tee >(jq >&2) \
    | jq '.response.upload_url'
)

{
  "response": {
    "upload_url": "https://pu.vk.com/c909618/upload_doc.php?act=add_doc&mid=329361482&aid=0&gid=0&type=0&hash=907a7c32a34e65d46252e34a749d1fc2&rhash=ebc8504b1dea2ed80e8387a58776d3f6&api=1"
  }
}


In [19]:
echo "$upload_url"

"https://pu.vk.com/c909618/upload_doc.php?act=add_doc&mid=329361482&aid=0&gid=0&type=0&hash=907a7c32a34e65d46252e34a749d1fc2&rhash=ebc8504b1dea2ed80e8387a58776d3f6&api=1"


In [20]:
file_info=$(
    curl -s -X POST ${upload_url:1:-1} -F file=@./lab3.ipynb \
    | tee >(jq >&2) \
    | jq '.file'
)

{
  "file": "329361482|0|0|909618|37cd7d427e|ipynb|73295|lab3.ipynb|c2c516118fbcefdf02dc13528af9c675|4406a5e620e50aac148805474b8816c6||||eyJkaXNrIjoiNTcifQ=="
}


In [21]:
echo "$file_info"

"329361482|0|0|909618|37cd7d427e|ipynb|73295|lab3.ipynb|c2c516118fbcefdf02dc13528af9c675|4406a5e620e50aac148805474b8816c6||||eyJkaXNrIjoiNTcifQ=="


In [22]:
curl -Gs https://api.vk.com/method/docs.save \
-d v=5.199 \
-d access_token=$TOKEN \
-d file=${file_info:1:-1} \
-d title=$(openssl rand -base64 12)-lab3.ipynb | jq

{
  "response": {
    "type": "doc",
    "doc": {
      "id": 671974040,
      "owner_id": 329361482,
      "title": "hhGNbPGL5tnyV0be-lab3.ipynb",
      "size": 73295,
      "ext": "ipynb",
      "date": 1714644188,
      "type": 8,
      "url": "https://vk.com/doc329361482_671974040?hash=i2NseU84jNwVMNYzcg8YuZvatVJokLY2xkoZAMJv12k&dl=DcrGIAeMzLaaRgqv9nUgdzRZx8OI9aHt5MlojHUIqbE&api=1&no_preview=1",
      "is_unsafe": 0,
      "can_manage": true
    }
  }
}


## Группа методов `photos`

### Получим список всех фотографий с их размерами, а также скрытые (если они есть)

In [23]:
photo_id=$(
    curl -Gs https://api.vk.com/method/photos.getAll \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d owner_id=329361482 \
    -d need_hidden=1 \
    -d photo_sizes=1 \
    -d count=2 \
    | tee >(jq 'del(.response.items.[].sizes[0:-1])' >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 11,
    "items": [
      {
        "album_id": 302557033,
        "date": 1714610506,
        "id": 457241796,
        "owner_id": 329361482,
        "sizes": [
          {
            "height": 510,
            "type": "r",
            "width": 510,
            "url": "https://sun9-7.userapi.com/impg/vB4QPq5S8QbVpkmxyHwJ23v0k1N81Bkj8J9skQ/yWGBDi7PIW8.jpg?size=510x510&quality=96&sign=e62bd9316b919c6c35fd784b1b9d2eb8&c_uniq_tag=wFHV0eliNe_QaHsFtMf4BJ4LoRDqpUAy5ExP0JrzGpw&type=album"
          }
        ],
        "text": "pFaG/aUyy48ml6xP-418.jpg",
        "web_view_token": "4d0147ac538f09bc72",
        "has_tags": false,
        "hidden": 1
      },
      {
        "album_id": 302557033,
        "date": 1714610457,
        "id": 457241795,
        "owner_id": 329361482,
        "sizes": [
          {
            "height": 510,
            "type": "r",
            "width": 510,
            "url": "https://sun9-29.userapi.com/impg/42-Ka8J-nCeAv38QraDQD7ebij

In [24]:
echo "$photo_id"

457241796


### Комментарии к фото

In [25]:
curl -Gs https://api.vk.com/method/photos.getComments \
-d v=5.199 \
-d access_token=$TOKEN \
-d owner_id=329361482 \
-d photo_id=$photo_id | jq

{
  "response": {
    "count": 0,
    "items": []
  }
}


### Загрузка фото в альбом

Процесс точно такой же, как с загрузкой файлов. Разьве что больше мест куда можно загрузить фото: на стену, в альбом, в сообщение и так далее.

In [26]:
curl https://random.imagecdn.app/512/512 -L -o /tmp/img.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    80  100    80    0     0     13      0  0:00:06  0:00:05  0:00:01    25
100   207  100   207    0     0     34      0  0:00:06  0:00:06 --:--:--    34
100 47188  100 47188    0     0   5771      0  0:00:08  0:00:08 --:--:-- 35293


In [27]:
album_id=$(
    curl -Gs https://api.vk.com/method/photos.getAlbums \
    -d v=5.199 \
    -d access_token=$TOKEN \
    | tee >(jq >&2) \
    | jq '.response.items | map(select(.title == "photos")).[0].id'
)

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 302557033,
        "owner_id": 329361482,
        "size": 7,
        "title": "photos",
        "feed_disabled": 0,
        "feed_has_pinned": 0,
        "created": 1714605876,
        "description": "",
        "can_delete": true,
        "can_include_to_feed": false,
        "is_locked": true,
        "privacy_comment": {
          "category": "friends",
          "lists": {
            "allowed": [],
            "excluded": []
          },
          "owners": {
            "allowed": [],
            "excluded": []
          }
        },
        "privacy_view": {
          "category": "only_me",
          "lists": {
            "allowed": [],
            "excluded": []
          },
          "owners": {
            "allowed": [],
            "excluded": []
          }
        },
        "thumb_id": 457241796,
        "thumb_is_last": 1,
        "updated": 1714610506
      }
    ]
  }
}


In [28]:
echo "$album_id"

302557033


In [29]:
if [ "$album_id" = "null" ]; then
    album_id=$(
        curl -Gs https://api.vk.com/method/photos.createAlbum \
        -d v=5.199 \
        -d access_token=$TOKEN \
        -d title="photos" \
        -d privacy_view=only_me \
        | tee >(jq >&2) \
        | jq '.response.id'
    )
fi

In [30]:
echo "$album_id"

302557033


In [31]:
upload_url=$(
    curl -Gs https://api.vk.com/method/photos.getUploadServer \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d album_id=$album_id \
    | tee >(jq >&2) \
    | jq '.response.upload_url'
)

{
  "response": {
    "album_id": 302557033,
    "upload_url": "https://pu.vk.com/c844030/ss2071/upload.php?act=do_add&mid=329361482&aid=302557033&gid=0&hash=587c40245a4012ba67dba813ec446f31&rhash=d0ca5263e00e2bee50f9b00368054afa&swfupload=1&api=1",
    "user_id": 329361482
  }
}


In [32]:
echo "$upload_url"

"https://pu.vk.com/c844030/ss2071/upload.php?act=do_add&mid=329361482&aid=302557033&gid=0&hash=587c40245a4012ba67dba813ec446f31&rhash=d0ca5263e00e2bee50f9b00368054afa&swfupload=1&api=1"


In [33]:
photo_info=$(
    curl -s -X POST ${upload_url:1:-1} -F file=@/tmp/img.png \
    | tee >(jq >&2)
)

{
  "server": 844030,
  "photos_list": "[{\"markers_restarted\":true,\"photo\":\"8867633661:x\",\"sizes\":[],\"latitude\":0,\"longitude\":0,\"kid\":\"86082fe7db7fe7f73f01f82770700490\",\"sizes2\":[[\"s\",\"4cb0c548a2a811e60c535cf510553a159898bafcec85eb33be17e4b6\",\"6885674012536534566\",75,75],[\"m\",\"004063aee3ca05592c9b2fafd62041d96869892fb58b8f4d9e79480b\",\"-639396101755076902\",130,130],[\"x\",\"4dbb680b47f360a9a4663904631936f6a82269241ee9b206b06261d3\",\"2011052438627191579\",512,512],[\"o\",\"004063aee3ca05592c9b2fafd62041d96869892fb58b8f4d9e79480b\",\"-9150484754182156995\",130,130],[\"p\",\"3566b2de12c3c4591a429f8cf25ad7444709f97a316d47350c93c238\",\"-7898201955223191673\",200,200],[\"q\",\"cfe8af9c4894f65954ebbfcd1abec5cca95bfd7a2eabbcd28e1d7b6d\",\"715291089221660895\",320,320],[\"r\",\"9ebbf8c6aabbb346661bf2cb6cbcc8b9ef8cb1ff056999ffbf2a824b\",\"1115762380967512657\",510,510]],\"urls\":[],\"urls2\":[\"TLDFSKKoEeYMU1z1EFU6FZiYuvzsheszvhfktg/JvZgVAjUjl8.jpg\",\"AEBjruPKBVks

In [34]:
server=$(echo "$photo_info" | jq '.server')
photos_list=$(echo "$photo_info" | jq '.photos_list' | sed 's/\\\"/"/g')
hash=$(echo "$photo_info" | jq '.hash')
echo "${hash:1:-1}"

11cd956f03a1962d07f934e9e981c0ad


In [35]:
curl -Gs https://api.vk.com/method/photos.save \
-d v=5.199 \
-d access_token=$TOKEN \
-d album_id=$album_id \
-d server=$server \
-d photos_list=${photos_list:1:-1} \
-d hash=${hash:1:-1} \
-d caption=$(openssl rand -base64 12)-418.jpg | jq

{
  "response": [
    {
      "album_id": 302557033,
      "date": 1714644202,
      "id": 457241797,
      "owner_id": 329361482,
      "sizes": [
        {
          "height": 75,
          "type": "s",
          "width": 75,
          "url": "https://sun9-67.userapi.com/impg/TbtoC0fzYKmkZjkEYxk29qgiaSQe6bIGsGJh0w/G4sOtYmx6Bs.jpg?size=75x75&quality=96&sign=3d0369c1d4a66005b4ac20556cc70ec8&c_uniq_tag=E4nATb161oyNJY07FCwCEvQDIitUv1Bbh0TspTRYjr0&type=album"
        },
        {
          "height": 130,
          "type": "m",
          "width": 130,
          "url": "https://sun9-67.userapi.com/impg/TbtoC0fzYKmkZjkEYxk29qgiaSQe6bIGsGJh0w/G4sOtYmx6Bs.jpg?size=130x130&quality=96&sign=6f4bcf6f06ebba10e1149354e88cd4e1&c_uniq_tag=_J0cHAHtO16sviG5f6yvdQphO7me554SrLKjo_6gDq8&type=album"
        },
        {
          "height": 512,
          "type": "x",
          "width": 512,
          "url": "https://sun9-67.userapi.com/impg/TbtoC0fzYKmkZjkEYxk29qgiaSQe6bIGsGJh0w/G4sOtYmx6Bs.jpg?size=512x512

# Вывод

В данной лабораторной работе мы познакомилось с API VK.